In [ ]:
import numpy as np

In [ ]:
# References:
# Kai Tao, Tianze Liu, Jieyuan Ning, Fenglin Niu, "Estimating sedimentary and crustal structure
# using wavefield continuation: theory, techniques and applications", Geophysical Journal International,
# Volume 197, Issue 1, April, 2014, Pages 443-457, https://doi.org/10.1093/gji/ggt515

In [ ]:
import sympy

In [ ]:
sympy.init_printing()

In [ ]:
sympy.var('α β p ρ μ ω', positive=True, real=True)

In [ ]:
sympy.var('z z0 t t0 t1', real=True)

In [ ]:
μ = ρ*β**2
μ

In [ ]:
q_α = sympy.sqrt(1/α**2 - p**2)
q_α

In [ ]:
q_β = sympy.sqrt(1/β**2 - p**2)
q_β

In [ ]:
η = 1/β**2 - 2*p**2
η

In [ ]:
M = sympy.Matrix([
    [         α*p,         α*p,       β*q_β,       β*q_β],
    [       α*q_α,      -α*q_α,        -β*p,         β*p],
    [-2*α*μ*p*q_α, 2*α*μ*p*q_α,      -β*μ*η,       β*μ*η],
    [      -α*μ*η,      -α*μ*η, 2*β*μ*p*q_β, 2*β*μ*p*q_β]
])
M

In [ ]:
Minv = (1/ρ)*sympy.Matrix([
    [        μ*p/α,  η*μ/(2*α*q_α), -p/(2*α*q_α),    -1/(2*α)],
    [        μ*p/α, -η*μ/(2*α*q_α),  p/(2*α*q_α),    -1/(2*α)],
    [η*μ/(2*β*q_β),         -μ*p/β,     -1/(2*β), p/(2*β*q_β)],
    [η*μ/(2*β*q_β),          μ*p/β,      1/(2*β), p/(2*β*q_β)]
])
Minv

In [ ]:
deig = sympy.diag(-q_α, q_α, -q_β, q_β)
deig

In [ ]:
# Lame constant
λ = ρ*(α**2 - 2*β**2)
λ

In [ ]:
γ = 4*μ*(λ + μ)/(λ + 2*μ)
γ

In [ ]:
A = sympy.Matrix([
    [0, p, 1/μ, 0],
    [p*λ/(λ + 2*μ), 0, 0, 1/(λ + 2*μ)],
    [ρ - (p**2)*γ, 0, 0, p*λ/(λ + 2*μ)],
    [0, ρ, p, 0]
])
A

In [ ]:
sympy.simplify(A - M*deig*Minv)

In [ ]:
Pdiag = sympy.diag(sympy.exp(sympy.I*ω*q_α*(z - z0)),
                   sympy.exp(-sympy.I*ω*q_α*(z - z0)),
                   sympy.exp(sympy.I*ω*q_β*(z - z0)),
                   sympy.exp(-sympy.I*ω*q_β*(z - z0)))
Pdiag

In [ ]:
P = M*Pdiag*Minv

## Study case of half-space with no layers (mantle only)

In [ ]:
sympy.var('v_r0 v_z0')

In [ ]:
P0 = P.subs([(z0, 0)])

In [ ]:
w = Minv*P0*sympy.Matrix([v_r0, v_z0, 0, 0])

In [ ]:
S_up = sympy.simplify(w[3])

In [ ]:
St_up = sympy.InverseFourierTransform(S_up, ω, t)

In [ ]:
St_up

In [ ]:
Jz_Sup = -ρ*(β**2)*q_β*sympy.Abs(St_up)**2

In [ ]:
Jz_Sup

In [ ]:
energy = sympy.Integral(Jz_Sup, (t, t0, t1))

In [ ]:
energy

In [ ]:
# Units of density don't matter since it turns out to be just a scaling factor

--------------------------------------------------------

# Load some data and start computing cost function

In [ ]:
from collections import defaultdict
import logging

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import numpy.fft as fft

import h5py
import obspy
import obspyh5

from seismic.receiver_fn.stream_quality_filter import curate_stream3c
from seismic.receiver_fn.rf_util import compute_vertical_snr
from seismic.receiver_fn.rf_util import KM_PER_DEG

## Test first on non-sedimentary station since the theory is supposed to still work

In [ ]:
network = 'OA'
target_station = 'BT23'

In [ ]:
# Resampling rate
f_s = 10.0
# Time window of original data to use for processing. All traces must have at least this extent
# about the onset time.
TIME_WINDOW = (-20, 60)
# Narrower time window used for integration of energy flux
FLUX_WINDOW = (-10, 20)

In [ ]:
src_file = r"/g/data1a/ha3/am7399/shared/OA_RF_analysis/OA_event_waveforms_for_rf_20170911T000036-20181128T230620_rev8.h5"

In [ ]:
traces = []
for tr in obspyh5.iterh5(src_file, group='/waveforms/{}.{}.0M'.format(network, target_station), mode='r'):
    traces.append(tr)

In [ ]:
# Group triplets of traces for same event id
data_all = defaultdict(obspy.Stream)
for tr in traces:
    data_all[tr.stats.event_id].append(tr.copy())

In [ ]:
# Re-order traces into ZNE order.
# Trim traces to analysis time window.
for evid, stream in data_all.items():
    stream.trim(stream[0].stats.onset + TIME_WINDOW[0],
                stream[0].stats.onset + TIME_WINDOW[1])
    stream.sort(keys=['channel'], reverse=True)

In [ ]:
len(data_all)

In [ ]:
data_all[list(data_all.keys())[0]]

In [ ]:
# Apply curation to streams prior to rotation
logger = logging.getLogger(__name__)
discard_ids = []
for evid, stream in data_all.items():
    if not curate_stream3c(evid, stream, logger):
        discard_ids.append(evid)

for evid in discard_ids:
    data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
# Rotate to ZRT coordinates
for evid, stream in data_all.items():
    stream.rotate('NE->RT')

In [ ]:
sample_data = data_all[list(data_all.keys())[0]]
sample_data

In [ ]:
sample_data.plot()

In [ ]:
# Detrend and taper the traces
for evid, stream in data_all.items():
    stream.detrend('linear')
    stream.taper(0.05)

In [ ]:
sample_data.plot()

In [ ]:
sample_data[0].stats

In [ ]:
# Resample data to lower sampling rate
for evid, stream in data_all.items():
    stream.filter('bandpass', freqmin=0.02, freqmax=f_s/2.0, corners=2, zerophase=True).interpolate(f_s, method='lanczos', a=10)

In [ ]:
sample_data.plot()

In [ ]:
# Compute SNR of Z component to use as a quality metric
for evid, stream in data_all.items():
    compute_vertical_snr(stream)

In [ ]:
sample_data[0].stats

In [ ]:
snrs = np.array([s[0].stats.snr_prior for _, s in data_all.items()])

In [ ]:
plt.hist(snrs, bins=np.linspace(0, 10, 21))
plt.show()

In [ ]:
discard_ids = []
for evid, stream in data_all.items():
    if stream[0].stats.snr_prior < 2.0:
        discard_ids.append(evid)
        
for evid in discard_ids:
    data_all.pop(evid)

In [ ]:
len(data_all)

In [ ]:
sample_data = data_all[list(data_all.keys())[0]]
sample_data.plot()

In [ ]:
assert(np.all(np.array([tr.stats.npts for st in data_all.values() for tr in st]) == 
              data_all[list(data_all.keys())[0]][0].stats.npts))

## Start processing

In [ ]:
# TODO: Update this to use dask instead of numpy, so that results will be computed lazily.

In [ ]:
dt = 1.0/f_s
npts = data_all[list(data_all.keys())[0]][0].stats.npts
nevts = len(data_all)

In [ ]:
t = np.linspace(*TIME_WINDOW, npts)

In [ ]:
# Extract time series for Vr and Vz from data_all and shape into 3D array. First row is Vr, second row is Vz.
# The time series is in the second (column) direction.
# The third (depth) dimension is the number of events. Each depth plane is Vr, Vz time series for one event.


In [ ]:
# Note here that we negate the z-component, since this method as +z as downwards (increasing depth).
# V0 represents P-SV signal at the surface, i.e. that recorded by surface seismometer.
v0 = np.array([[st[1].data.tolist(), (-st[0].data).tolist()] for st in data_all.values()])

In [ ]:
v0.shape

In [ ]:
v0 = np.moveaxis(v0, 0, -1)
v0.shape

In [ ]:
# Normalize each event signal by the maximum z-component amplitude.
# We perform this succinctly using numpy multidimensional broadcasting rules.
max_vz = v0[1,:,:].max(axis=0)
max_vz.shape

In [ ]:
v0 = v0/max_vz

In [ ]:
# Transform v0 to the spectral domain using real FFT
fv0 = np.fft.rfft(v0, axis=1)

In [ ]:
fv0.shape

In [ ]:
# Compute discrete frequencies
w = 2*np.pi*np.fft.rfftfreq(v0.shape[1], dt)
w.shape

In [ ]:
# Extend w to full spectral domain.
w_full = np.hstack((w, -np.flipud(w[1:])))
w_full.shape

In [ ]:
# To extend fv0, we need to flip left-right and take complex conjugate.
fv0_full = np.hstack((fv0, np.fliplr(np.conj(fv0[:, 1:, :]))))
fv0_full.shape

In [ ]:
fv0_full = np.moveaxis(fv0_full, -1, 0)
fv0_full.shape

In [ ]:
# Get ray params for the events
p = np.array([st[0].stats.slowness/KM_PER_DEG for st in data_all.values()])
p.shape

In [ ]:
def mode_matrices(Vp, Vs, rho, p):
    """Compute M, M_inv and Q for a single layer for a scalar or array of ray parameters p.
    
    Vp: alpha (scalar)
    Vs: beta (scalar)
    rho: rho (scalar)
    p: scalar or array of ray parameters (one per event)
    """
    qa = np.sqrt(1/Vp**2 - p*p)
    qb = np.sqrt(1/Vs**2 - p*p)
    eta = 1/Vs**2 - 2*p*p
    mu = rho*Vs*Vs
    trp = 2*mu*p*qa
    trs = 2*mu*p*qb
    mu_eta = mu*eta
    # First compute without velocity factors for reduced operation count.
    M = np.array([
        [p, p, qb, qb],
        [qa, -qa, -p, p],
        [-trp, trp, -mu_eta, mu_eta],
        [-mu_eta, -mu_eta, trs, trs]
    ])
    # Then times by velocity factors
    Vfactors = np.diag([Vp, Vp, Vs, Vs])
    M = np.matmul(np.moveaxis(M, -1, 0), Vfactors)
    
    Q = np.diag([qa, -qa, qb, -qb])

    # First compute without velocity factors for reduced operation count.
    mu_p = mu*p
    Minv = (1.0/rho)*np.array([
        [mu_p, mu_eta/2/qa, -p/2/qa, -0.5*np.ones(p.shape)],
        [mu_p, -mu_eta/2/qa, p/2/qa, -0.5*np.ones(p.shape)],
        [mu_eta/2/qb, -mu_p, -0.5*np.ones(p.shape), p/2/qb],
        [mu_eta/2/qb, mu_p, 0.5*np.ones(p.shape), p/2/qb]
    ])
    # Then times by velocity factors
    Vfactors_inv = np.diag([1/Vp, 1/Vp, 1/Vs, 1/Vs])
    Minv = np.matmul(Vfactors_inv, np.moveaxis(Minv, -1, 0))
    
    return (M, Minv, Q)

In [ ]:
Vp = 6.4
Vs = 3.7
rho = 2.7

In [ ]:
M, Minv, Q = mode_matrices(Vp, Vs, rho, p)

In [ ]:
M.shape

In [ ]:
Minv.shape

In [ ]:
Q.shape

In [ ]:
fv0_full.shape

In [ ]:
w_full.shape

In [ ]:
def propagate_layers(fv0, w, H, Vp, Vs, rho, p):
    nlayers = len(Vp)
    assert len(Vs) == len(rho) == len(H) == nlayers
    fz = fv0
    for i in range(nlayers):
        M, Minv, Q = mode_matrices(Vp[i], Vs[i], rho[i], p)
        fz = np.matmul(Minv[:,:,0:2], fz)
        phase_args = np.outer(Q - Q[1], w)
        phase_factors = np.exp(-1j*H[i]*phase_args) # shouldn't this H[i] rather be H[i] - H[i-1]?
        fz = fz*phase_factors
        fz = np.matmul(M, fz)
    # end for
    return fz

In [ ]:
fz = np.matmul(Minv[:,:,0:2], fv0_full)
fz.shape

In [ ]:
phase_args = np.outer(Q - Q[1], w_full)
phase_args.shape

In [ ]:
H = 35

In [ ]:
phase_factors = np.exp(-1j*H*phase_args)
phase_factors.shape

In [ ]:
fz = fz*phase_factors
fz.shape

In [ ]:
M.shape

In [ ]:
fz = np.matmul(M, fz)
fz.shape